<a href="https://colab.research.google.com/github/xiaofanpaiooo-code/whisper-lora/blob/xiaofanpaiooo-code-patch-1/whisper%E7%BD%91%E7%BB%9C%E7%AB%AF%E4%BB%A3%E7%A0%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==========================================
# 部署说明：
# 1. 将此代码块完整复制到 Google Colab 的一个单元格中。
# 2. 修改 NGROK_AUTH_TOKEN 为你的真实 Token (从 https://dashboard.ngrok.com/ 获取)。
# 3. 点击运行，等待出现 "Public URL" 即为接口地址。
# ==========================================

import os
import sys
import subprocess

# 1. 环境安装 (第一次运行时会自动安装)
def install_dependencies():
    # 新增 nest_asyncio 以解决 Colab 事件循环冲突
    packages = ["faster-whisper", "fastapi", "uvicorn", "python-multipart", "pyngrok", "nest_asyncio"]
    subprocess.check_call([sys.executable, "-m", "pip", "install"] + packages)

try:
    import faster_whisper
    import fastapi
    import nest_asyncio
except ImportError:
    print("正在安装依赖环境，请稍候...")
    install_dependencies()
    print("环境安装完成！")
    import nest_asyncio

# 应用 nest_asyncio 补丁 (关键修复：允许在 Colab 中运行 Uvicorn)
nest_asyncio.apply()

# 2. 导入库
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
from faster_whisper import WhisperModel
from pyngrok import ngrok
import uvicorn
import shutil
import time

# ================= 配置区域 =================
# TODO: 请在此处填入你的 Ngrok Authtoken
NGROK_AUTH_TOKEN = "37hXaj4l3VhFmrso4FTbEWYE3Li_7Z3vCLhdMAPb2BpiVufAw"
# ===========================================

# 3. 初始化 FastAPI 和 模型
app = FastAPI()

print("正在加载 Whisper 模型 (Tiny + float16)...")
# 关键约束：使用 faster-whisper, tiny模型, cuda加速, float16精度
model = WhisperModel("tiny", device="cuda", compute_type="float16")
print("模型加载完毕！")

@app.post("/transcribe")
async def transcribe(file: UploadFile = File(...)):
    """
    接收音频文件 -> 极速推理 -> 返回文本
    """
    start_time = time.time()
    temp_filename = f"temp_{file.filename}"

    try:
        # 保存上传的音频流到临时文件
        with open(temp_filename, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)

        # 推理核心
        # beam_size=1 牺牲微小的准确度换取最快速度
        segments, info = model.transcribe(temp_filename, beam_size=1, language="zh")

        text_result = "".join([segment.text for segment in segments])

        process_time = time.time() - start_time
        print(f"处理耗时: {process_time:.4f}s | 结果: {text_result}")

        return JSONResponse(content={"text": text_result.strip()})

    except Exception as e:
        return JSONResponse(content={"error": str(e)}, status_code=500)
    finally:
        # 清理临时文件
        if os.path.exists(temp_filename):
            os.remove(temp_filename)

# 4. 启动服务
if __name__ == "__main__":
    if not NGROK_AUTH_TOKEN or NGROK_AUTH_TOKEN == "在此处粘贴你的_NGROK_TOKEN":
        print("【错误】请设置 Ngrok Auth Token，否则无法穿透内网！")
    else:
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        # 开启 HTTP 隧道，指向 8000 端口
        # 注意：Colab 环境下有时候 connect 需要一点时间，或者需要先 kill 掉旧进程
        try:
            public_url = ngrok.connect(8000).public_url
            print(f"\n======== 服务已启动 ========")
            print(f"API 接口地址 (请复制到客户端): {public_url}/transcribe")
            print(f"===========================\n")

            # 使用 uvicorn.run 启动
            uvicorn.run(app, host="0.0.0.0", port=8000)
        except Exception as e:
            print(f"启动出错: {e}")
            print("提示：如果遇到端口占用，请尝试重启 Colab 运行时 (Runtime -> Restart session)")

正在安装依赖环境，请稍候...
环境安装完成！
正在加载 Whisper 模型 (Tiny + float16)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocabulary.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/75.5M [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

模型加载完毕！

======== 服务已启动 ========
API 接口地址 (请复制到客户端): https://unavoidable-amber-plinthlike.ngrok-free.dev/transcribe

启动出错: _patch_asyncio.<locals>.run() got an unexpected keyword argument 'loop_factory'
提示：如果遇到端口占用，请尝试重启 Colab 运行时 (Runtime -> Restart session)


/usr/local/lib/python3.12/dist-packages/uvicorn/server.py:67: RuntimeWarning: coroutine 'Server.serve' was never awaited
  return asyncio_run(self.serve(sockets=sockets), loop_factory=self.config.get_loop_factory())
